Imports

In [141]:
import numpy as np
!python --version
import matplotlib
import matplotlib.pyplot
import pandas as pd
import torch
import os
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy

#from PIL import Image

Python 3.10.6


Functions defined

In [142]:
import sys


def load_custom_dataset():
    paths = []
    testpaths = []
    testlabels = []
    terminator = 'D:/Uni/19.Master/Daten/terminator.wav'
    print(sys.executable)
    emotions = []
    # for dirname, _, filenames in os.walk('Daten/TESS Toronto emotional speech set data'):
    # D:\Uni\19.Master\DATEN
    for dirname, _, filenames in os.walk('../tess'):
        for filename in filenames:
            label = filename.split('_')[-1]
            label = label.split('.')[0]
            if (label != 'neutral'):
                emotions.append(label.lower())
                paths.append(os.path.join(dirname, filename))
    for dirname, _, filenames in os.walk('../stimuli_intensitätsmorphs'):
        for filename in filenames:

            intens = filename.split('_')[-2]
            emot = filename.split('_')[1]
            label = emot
            match label:
                case 'ang':
                    label = 'angry'
                case 'dis':
                    label = 'disgust'
                case 'fea':
                    label = 'fear'
                case 'hap':
                    label = 'happy'
                case 'sad':
                    label = 'sad'
                case 'sur':
                    label = 'ps'
            if (emot != 'ple'):
                testpaths.append(os.path.join(dirname, filename))
                testlabels.append(label.lower())
    com_labels = testlabels + emotions
    com_paths = testpaths + paths
    print(testlabels)
    print(testpaths)
    print('Dataset is loaded')
    return paths, emotions, testpaths, testlabels
"""
def scale_minmax(X, min=0.0, max=1.0):
    X_std = (X - X.min()) / (X.max() - X.min())
    X_scaled = X_std * (max - min) + min
    return X_scaled

def spectrogram_image(y, sr, out, hop_length, n_mels):
    # use log-melspectrogram
    mels = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels,
                                            n_fft=hop_length*2, hop_length=hop_length)
    mels = numpy.log(mels + 1e-9) # add small number to avoid log(0)

    # min-max scale to fit inside 8-bit range
    img = scale_minmax(mels, 0, 255).astype(numpy.uint8)
    img = numpy.flip(img, axis=0) # put low frequencies at the bottom in image
    img = 255-img # invert. make black==more energy

    # save as PNG
    #plt.axis('off')
    plt.imsave("spec.png",img)
    plt.imshow(img)

def createSpectrogramm(path, label):
    data, sr = librosa.load(path)
    fourier = librosa.stft(data)
    fourierdb = librosa.amplitude_to_db(abs(fourier))
    librosa.display.specshow(fourierdb, sr=sr, x_axis='time', y_axis='hz')

def createMelSpectrogramm(path, label):
    data, sr = librosa.load(path, sr=22050, offset=0, duration=1)
    melSpec = librosa.feature.melspectrogram(data, sr, n_mels=192, n_fft=1024, hop_length=260)

    # plt.figure(figsize=(10, 4))
    # plt.title(label, size=20)
    librosa.display.specshow(melSpec, sr=sr, x_axis='time', y_axis='hz')
    # plt.colorbar()
"""

'\ndef scale_minmax(X, min=0.0, max=1.0):\n    X_std = (X - X.min()) / (X.max() - X.min())\n    X_scaled = X_std * (max - min) + min\n    return X_scaled\n\ndef spectrogram_image(y, sr, out, hop_length, n_mels):\n    # use log-melspectrogram\n    mels = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels,\n                                            n_fft=hop_length*2, hop_length=hop_length)\n    mels = numpy.log(mels + 1e-9) # add small number to avoid log(0)\n\n    # min-max scale to fit inside 8-bit range\n    img = scale_minmax(mels, 0, 255).astype(numpy.uint8)\n    img = numpy.flip(img, axis=0) # put low frequencies at the bottom in image\n    img = 255-img # invert. make black==more energy\n\n    # save as PNG\n    #plt.axis(\'off\')\n    plt.imsave("spec.png",img)\n    plt.imshow(img)\n\ndef createSpectrogramm(path, label):\n    data, sr = librosa.load(path)\n    fourier = librosa.stft(data)\n    fourierdb = librosa.amplitude_to_db(abs(fourier))\n    librosa.display.specsho

In [143]:
trainpaths, trainlabels, testpaths, testlabels = load_custom_dataset()

###create dataframes for training and testing###
trainDF = pd.DataFrame()
trainDF["path"] = trainpaths
trainDF["emotion"] = trainlabels

testDF = pd.DataFrame()
testDF["path"] = testpaths
testDF["emotion"] = testlabels


testDF

X:\masterarbeit_programming\venv\Scripts\python.exe
['angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'disgust', 'disgust', 'disgust', 'disgust', 'disgust', 'disgust', 'disgust', 'disgust', 'disgust', 'disgust', 'disgust', 'disgust', 'disgust', 'disgust', 'disgust', 'disgust', 'fear', 'fear', 'fear', 'fear', 'fear', 'fear', 'fear', 'fear', 'fear', 'fear', 'fear', 'fear', 'fear', 'fear', 'fear', 'fear', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'happy', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'sad', 'ps', 'ps', 'ps', 'ps', 'ps', 'ps', 'ps', 'ps', 'ps', 'ps', 'ps', 'ps', 'ps', 'ps', 'ps', 'ps', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'angry', 'di

,path,emotion
0,../stimuli_intensitätsmorphs\nf01_ang_w01_o_10...,angry
1,../stimuli_intensitätsmorphs\nf01_ang_w01_o_25...,angry
2,../stimuli_intensitätsmorphs\nf01_ang_w01_o_50...,angry
3,../stimuli_intensitätsmorphs\nf01_ang_w01_o_75...,angry
4,../stimuli_intensitätsmorphs\nf01_ang_w02_o_10...,angry
...,...,...
763,../stimuli_intensitätsmorphs\nm04_sur_w03_o_75...,ps
764,../stimuli_intensitätsmorphs\nm04_sur_w05_o_10...,ps
765,../stimuli_intensitätsmorphs\nm04_sur_w05_o_25...,ps
766,../stimuli_intensitätsmorphs\nm04_sur_w05_o_50...,ps


In [144]:
"""
path = trainDF["speech"][0]
# settings
hop_length = 302 # number of samples per time-step in spectrogram
n_mels = 192 # number of bins in spectrogram. Height of image
time_steps = 192 # number of time-steps. Width of image

# load audio. Using example from librosa
y, sr = librosa.load(path, offset=0.1, duration=1, sr=99100)
out = 'out.png'

# extract a fixed length window
start_sample = 0 # starting at beginning
length_samples = time_steps*hop_length
window = y[start_sample:start_sample+length_samples]

# convert to PNG
spectrogram_image(window, sr=sr, out=out, hop_length=hop_length, n_mels=n_mels)

#train_spectros = trainDF["speech"].apply(lambda x: spectrogram_image(x))
"""

'\npath = trainDF["speech"][0]\n# settings\nhop_length = 302 # number of samples per time-step in spectrogram\nn_mels = 192 # number of bins in spectrogram. Height of image\ntime_steps = 192 # number of time-steps. Width of image\n\n# load audio. Using example from librosa\ny, sr = librosa.load(path, offset=0.1, duration=1, sr=99100)\nout = \'out.png\'\n\n# extract a fixed length window\nstart_sample = 0 # starting at beginning\nlength_samples = time_steps*hop_length\nwindow = y[start_sample:start_sample+length_samples]\n\n# convert to PNG\nspectrogram_image(window, sr=sr, out=out, hop_length=hop_length, n_mels=n_mels)\n\n#train_spectros = trainDF["speech"].apply(lambda x: spectrogram_image(x))\n'

In [145]:

from torchvision.models import AlexNet_Weights
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
"""
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
"""

'\nprocessor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")\nmodel = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")\n'

In [146]:
from datasets import load_dataset
import wer
import torch
"""
# load dummy dataset and read soundfiles
ds = load_dataset("patrickvonplaten/librispeech_asr_dummy", "clean", split="validation")
# tokenize
input_values = processor(ds[0]["audio"]["array"], return_tensors="pt", padding="longest").input_values  # Batch size 1

# retrieve logits
logits = model(input_values).logits

# take argmax and decode
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)
"""

'\n# load dummy dataset and read soundfiles\nds = load_dataset("patrickvonplaten/librispeech_asr_dummy", "clean", split="validation")\n# tokenize\ninput_values = processor(ds[0]["audio"]["array"], return_tensors="pt", padding="longest").input_values  # Batch size 1\n\n# retrieve logits\nlogits = model(input_values).logits\n\n# take argmax and decode\npredicted_ids = torch.argmax(logits, dim=-1)\ntranscription = processor.batch_decode(predicted_ids)\n'

In [147]:
"""
y = librosa.load(sr=16000, path=testDF["speech"][0])


input_values = processor(y[0], return_tensors="pt",  sampling_rate=16000).input_values  # Batch size 1

# retrieve logits
logits = model(input_values).logits

# take argmax and decode
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.batch_decode(predicted_ids)
"""

'\ny = librosa.load(sr=16000, path=testDF["speech"][0])\n\n\ninput_values = processor(y[0], return_tensors="pt",  sampling_rate=16000).input_values  # Batch size 1\n\n# retrieve logits\nlogits = model(input_values).logits\n\n# take argmax and decode\npredicted_ids = torch.argmax(logits, dim=-1)\ntranscription = processor.batch_decode(predicted_ids)\n'

In [148]:
"""
train_audios=[]
for x in testDF["speech"]:
    audio=librosa.load(path=x, sr=16000)
    train_audios.append(audio[0])
train_audios
"""

'\ntrain_audios=[]\nfor x in testDF["speech"]:\n    audio=librosa.load(path=x, sr=16000)\n    train_audios.append(audio[0])\ntrain_audios\n'

In [149]:
from sklearn.model_selection import train_test_split
save_path="/masterarbeit_programming/notebooks/content/data"
train_df, test_df=train_test_split(trainDF, test_size=0.2, random_state=101, stratify=trainDF["emotion"])

train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

train_df.to_csv(f"{save_path}/train.csv", sep="\t", encoding="utf-8", index=False)
test_df.to_csv(f"{save_path}/test.csv", sep="\t", encoding="utf-8", index=False)

In [150]:
print(train_df.shape)
print(test_df.shape)

(1920, 2)
(480, 2)


In [151]:
# Loading the created dataset using datasets
from datasets import load_dataset, load_metric

data_files = {
    "train": "/masterarbeit_programming/notebooks/content/data/train.csv",
    "validation": "/masterarbeit_programming/notebooks/content/data/test.csv",
}

dataset = load_dataset("csv", data_files=data_files, delimiter="\t", )
train_df = dataset["train"]
test_df = dataset["validation"]

print(train_df)
print(test_df)


Using custom data configuration default-235da878ddab3f84


Generating train split: 0 examples [00:00, ? examples/s]

X:\masterarbeit_programming\venv\lib\site-packages\datasets\download\streaming_download_manager.py:714: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)


Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to C:/Users/tonib/.cache/huggingface/datasets/csv/default-235da878ddab3f84/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


X:\masterarbeit_programming\venv\lib\site-packages\datasets\download\streaming_download_manager.py:714: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)


  0%|          | 0/2 [00:00<?, ?it/s]

Dataset({
    features: ['path', 'emotion'],
    num_rows: 1920
})
Dataset({
    features: ['path', 'emotion'],
    num_rows: 480
})


In [152]:
# specifiy input and output column
input_colum="path"
output_column="emotion"

In [153]:

#distinguish labels
class_list = train_df.unique(output_column)
class_list.sort()
num_class = len(class_list)

print(f"{num_class} Classes: {class_list}")

6 Classes: ['angry', 'disgust', 'fear', 'happy', 'ps', 'sad']


In [154]:
from transformers import AutoConfig, Wav2Vec2Processor

model_name_path =  "facebook/wav2vec2-base-960h"
pooling_mode = "mean"

In [155]:
### config
config = AutoConfig.from_pretrained(
    model_name_path,
    num_labels=num_class,
    label2id={label: i for i, label in enumerate(class_list)},
    id2label={i: label for i, label in enumerate(class_list)},
    finetuning_task="wav2vec2_clf"
)
setattr(config, 'pooling_mode', pooling_mode)

loading configuration file config.json from cache at C:\Users\tonib/.cache\huggingface\hub\models--facebook--wav2vec2-base-960h\snapshots\22aad52d435eb6dbaf354bdad9b0da84ce7d6156\config.json
Model config Wav2Vec2Config {
  "_name_or_path": "facebook/wav2vec2-base-960h",
  "activation_dropout": 0.1,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForCTC"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 256,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": false,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "sum",
  "ctc_zero_infinity": false,
  "diversity_loss_weight": 0.1,
  "do_stable_layer_norm": false,
  "eos_token_id": 2,
  "fe

In [156]:
print(config)
processor = Wav2Vec2Processor.from_pretrained(model_name_path)
target_sampling_rate = processor.feature_extractor.sampling_rate
print(f"Target sample rate: {target_sampling_rate}")

Wav2Vec2Config {
  "_name_or_path": "facebook/wav2vec2-base-960h",
  "activation_dropout": 0.1,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForCTC"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 256,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": false,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "sum",
  "ctc_zero_infinity": false,
  "diversity_loss_weight": 0.1,
  "do_stable_layer_norm": false,
  "eos_token_id": 2,
  "feat_extract_activation": "gelu",
  "feat_extract_dropout": 0.0,
  "feat_extract_norm": "group",
  "feat_proj_dropout": 0.1,
  "feat_quantizer_dropout": 0.0,
  "final_dropout": 0.1,
  "finetuning_task": "wa

loading configuration file preprocessor_config.json from cache at C:\Users\tonib/.cache\huggingface\hub\models--facebook--wav2vec2-base-960h\snapshots\22aad52d435eb6dbaf354bdad9b0da84ce7d6156\preprocessor_config.json
Feature extractor Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": false,
  "sampling_rate": 16000
}

loading configuration file config.json from cache at C:\Users\tonib/.cache\huggingface\hub\models--facebook--wav2vec2-base-960h\snapshots\22aad52d435eb6dbaf354bdad9b0da84ce7d6156\config.json
Model config Wav2Vec2Config {
  "_name_or_path": "facebook/wav2vec2-base-960h",
  "activation_dropout": 0.1,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForCTC"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 1,
  "classifier_proj_size

Target sample rate: 16000


### Preprocessing Data

In [157]:
# import torch
# import torchaudio
# print(torchaudio.__version__)
# print(torch.__version__)
# torchaudio.load(train_df[0]['path']['path'])
# train_df[0]['path']['path']

In [189]:
import torchaudio
import soundfile

def speech_file_to_array(speech_path):

    speech_array, sampling_rate = torchaudio.load(speech_path, normalize=True)
    print('test1')
    resampler = torchaudio.transforms.Resample(sampling_rate, target_sampling_rate)
    print('test2')
    speech = resampler(speech_array).squeeze().numpy()
    print('test3')
    return speech

def speech_file_to_array_librosa(speech_path):
    speech_array, sampling_rate = librosa.load(speech_path)
    resampler = librosa.resample(speech_array, orig_sr=sampling_rate, target_sr=target_sampling_rate).squeeze()

    return resampler

def label_to_id(label, label_list):

    if len(label_list) > 0:
        return label_list.index(label) if label in label_list else -1
    return label

def preprocess_function(examples):
    speech_list = [ speech_file_to_array_librosa(speech_path) for speech_path in examples[input_colum]]
    target_list = [ label_to_id(label, class_list) for label in examples[output_column]]
    attention_mask_list= [numpy.ones(shape=sf.shape) for sf in speech_list]

    result = processor(speech_list, sampling_rate=target_sampling_rate)
    result["labels"] = list(target_list)
    result["attention_mask"] = list(attention_mask_list)
    return result

In [190]:
### resample
#from datasets import Audio
#train_df = train_df.cast_column("path", Audio(sampling_rate=target_sampling_rate))

#test_df = test_df.cast_column("path", Audio(sampling_rate=target_sampling_rate))



In [191]:
train_df = train_df.map(
    preprocess_function,
    batch_size=100,
    batched=True,
    #num_proc=4
)
test_df = test_df.map(
    preprocess_function,
    batch_size=100,
    batched=True,
    #num_proc=4
)

#test1 = speech_file_to_array(test_df[0]['path'])
from datasets import Audio
# train_df = train_df.cast_column('path', Audio(sampling_rate=16000))
# test_df = test_df.cast_column('path', Audio(sampling_rate=16000))
#test2 = speech_file_to_array_librosa(test_df[0]['path'])


  0%|          | 0/20 [00:00<?, ?ba/s]

X:\masterarbeit_programming\venv\lib\site-packages\transformers\feature_extraction_utils.py:164: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)


  0%|          | 0/5 [00:00<?, ?ba/s]

In [192]:
train_df[9]

{'path': '../tess\\OAF_Pleasant_surprise\\OAF_chief_ps.wav',
 'emotion': 'ps',
 'input_values': [0.00012689562572631985,
  0.0058380174450576305,
  0.00578723568469286,
  0.0047672828659415245,
  0.003909342456609011,
  0.004343182779848576,
  0.005542287603020668,
  0.005779862869530916,
  0.00740136718377471,
  0.010440682992339134,
  0.01527947373688221,
  0.017540067434310913,
  0.018711376935243607,
  0.022364282980561256,
  0.024427887052297592,
  0.021836576983332634,
  0.024900106713175774,
  0.02727128192782402,
  0.028622902929782867,
  0.032586876302957535,
  0.032769132405519485,
  0.03493119776248932,
  0.03769749402999878,
  0.03610454499721527,
  0.033735666424036026,
  0.03351058065891266,
  0.034070588648319244,
  0.03399619087576866,
  0.03080596588551998,
  0.0325588583946228,
  0.03685218468308449,
  0.03479083254933357,
  0.03440998122096062,
  0.034398630261421204,
  0.03036915697157383,
  0.028516311198472977,
  0.02781406044960022,
  0.029108155518770218,
  0.03

In [162]:
#from datasets import ClassLabel

### label 2 id

#train_df = train_df.cast_column(output_column, ClassLabel(num_classes=num_class,names=class_list))

#test_df = test_df.cast_column(output_column, ClassLabel(num_classes=num_class,names=class_list))

In [193]:
train_df[0]

{'path': '../tess\\OAF_Fear\\OAF_choice_fear.wav',
 'emotion': 'fear',
 'input_values': [0.004722138401120901,
  -0.00035432036384008825,
  0.0010949079878628254,
  -0.0003840071731247008,
  -0.0029296313878148794,
  -0.0010490970453247428,
  -0.0042106714099645615,
  -0.003468363545835018,
  -0.003068897407501936,
  -0.007702418137341738,
  -0.007001595105975866,
  -0.005584786646068096,
  -0.0098490035161376,
  -0.010392286814749241,
  -0.010619891807436943,
  -0.010391466319561005,
  -0.008252652361989021,
  -0.01080710906535387,
  -0.011785155162215233,
  -0.013369373977184296,
  -0.012864617630839348,
  -0.01156588364392519,
  -0.013559042476117611,
  -0.013344797305762768,
  -0.014262956567108631,
  -0.015416963025927544,
  -0.0178836677223444,
  -0.019587934017181396,
  -0.01886187121272087,
  -0.02166689932346344,
  -0.020970268175005913,
  -0.020148999989032745,
  -0.02249022014439106,
  -0.023697098717093468,
  -0.024010350927710533,
  -0.02196562848985195,
  -0.0233544148504

### Model

In [194]:
from dataclasses import dataclass
from typing import Optional, Tuple
import torch
from transformers.file_utils import ModelOutput


@dataclass
class SpeechClassifierOutput(ModelOutput):
    loss: Optional[torch.FloatTensor] = None
    logits: torch.FloatTensor = None
    hidden_states: Optional[Tuple[torch.FloatTensor]] = None
    attentions: Optional[Tuple[torch.FloatTensor]] = None

In [195]:
import torch
import torch.nn as nn
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss

from transformers.models.wav2vec2.modeling_wav2vec2 import (
    Wav2Vec2PreTrainedModel,
    Wav2Vec2Model
)


class Wav2Vec2ClassificationHead(nn.Module):
    """Head for wav2vec classification task."""

    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.dropout = nn.Dropout(config.final_dropout)
        self.out_proj = nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, features, **kwargs):
        x = features
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        return x


class Wav2Vec2ForSpeechClassification(Wav2Vec2PreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.pooling_mode = config.pooling_mode
        self.config = config

        self.wav2vec2 = Wav2Vec2Model(config)
        self.classifier = Wav2Vec2ClassificationHead(config)

        self.init_weights()

    def freeze_feature_extractor(self):
        self.wav2vec2.feature_extractor._freeze_parameters()

    def merged_strategy(
            self,
            hidden_states,
            mode="mean"
    ):
        if mode == "mean":
            outputs = torch.mean(hidden_states, dim=1)
        elif mode == "sum":
            outputs = torch.sum(hidden_states, dim=1)
        elif mode == "max":
            outputs = torch.max(hidden_states, dim=1)[0]
        else:
            raise Exception(
                "The pooling method hasn't been defined! Your pooling mode must be one of these ['mean', 'sum', 'max']")

        return outputs

    def forward(
            self,
            input_values,
            attention_mask=None,
            output_attentions=None,
            output_hidden_states=None,
            return_dict=None,
            labels=None,
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        outputs = self.wav2vec2(
            input_values,
            attention_mask=attention_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        hidden_states = outputs[0]
        hidden_states = self.merged_strategy(hidden_states, mode=self.pooling_mode)
        logits = self.classifier(hidden_states)

        loss = None
        if labels is not None:
            if self.config.problem_type is None:
                if self.num_labels == 1:
                    self.config.problem_type = "regression"
                elif self.num_labels > 1 and (labels.dtype == torch.long or labels.dtype == torch.int):
                    self.config.problem_type = "single_label_classification"
                else:
                    self.config.problem_type = "multi_label_classification"

            if self.config.problem_type == "regression":
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels)
            elif self.config.problem_type == "single_label_classification":
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            elif self.config.problem_type == "multi_label_classification":
                loss_fct = BCEWithLogitsLoss()
                loss = loss_fct(logits, labels)

        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return SpeechClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

### Training

In [196]:
from dataclasses import dataclass
from typing import Dict, List, Optional, Union
import torch

import transformers
from transformers import Wav2Vec2Processor


@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [feature["labels"] for feature in features]

        d_type = torch.long if isinstance(label_features[0], int) else torch.float

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        batch["labels"] = torch.tensor(label_features, dtype=d_type)

        return batch

In [197]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [198]:
is_regression = False

In [199]:
from transformers import EvalPrediction


def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = numpy.squeeze(preds) if is_regression else numpy.argmax(preds, axis=1)

    if is_regression:
        return {"mse": ((preds - p.label_ids) ** 2).mean().item()}
    else:
        return {"accuracy": (preds == p.label_ids).astype(numpy.float32).mean().item()}

In [200]:
model = Wav2Vec2ForSpeechClassification.from_pretrained(
    model_name_path,
    config=config,
)

loading weights file pytorch_model.bin from cache at C:\Users\tonib/.cache\huggingface\hub\models--facebook--wav2vec2-base-960h\snapshots\22aad52d435eb6dbaf354bdad9b0da84ce7d6156\pytorch_model.bin
Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForSpeechClassification: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Wav2Vec2ForSpeechClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForSpeechClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForSpeechClassification were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newl

In [201]:
model.freeze_feature_extractor()

In [202]:
#print(is_apex_available())

In [217]:
from transformers import TrainingArguments
import apex
from apex import amp

training_args = TrainingArguments(
    output_dir="/masterarbeit_programming/notebooks/content",
    # output_dir="/content/gdrive/MyDrive/wav2vec2-xlsr-greek-speech-emotion-recognition"
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    evaluation_strategy="steps",
    num_train_epochs=1.0,
    fp16=True,
    save_steps=10,
    eval_steps=10,
    logging_steps=10,
    learning_rate=1e-4,
    save_total_limit=2,
    label_names=["0","1","2","3","4","5"],
)


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [218]:
#install apex
#!pip install -v --no-cache-dir   ../apex
num_class

6

In [219]:
###import
# import tensorflow as tf
import torch
import cuda
import sys
import numba

print("Python version:", sys.version)
print("Cuda version", cuda.__version__)
print("Numba version:", numba.__version__)
print("Numpy version:", numpy.__version__)
print("Torch version: ", torch.__version__)



Python version: 3.10.6 (tags/v3.10.6:9c7b4bd, Aug  1 2022, 21:53:49) [MSC v.1932 64 bit (AMD64)]
Cuda version 11.8.1
Numba version: 0.56.4
Numpy version: 1.23.4
Torch version:  1.13.0+cu117


In [220]:
import torch
import apex
print("cuda",torch.cuda.is_available())
transformers.utils.is_apex_available()

cuda True


True

In [221]:
from typing import Any, Dict, Union


import torch
from packaging import version
from torch import nn

from transformers import (
    Trainer,
    is_apex_available,
)
if is_apex_available():
    from apex import amp

if version.parse(torch.__version__) >= version.parse("1.6"):
    _is_native_amp_available = True
    from torch.cuda.amp import autocast


class CTCTrainer(Trainer):
    def training_step(self, model: nn.Module, inputs: Dict[str, Union[torch.Tensor, Any]]) -> torch.Tensor:
        """
        Perform a training step on a batch of inputs.

        Subclass and override to inject custom behavior.

        Args:
            model (:obj:`nn.Module`):
                The model to train.
            inputs (:obj:`Dict[str, Union[torch.Tensor, Any]]`):
                The inputs and targets of the model.

                The dictionary will be unpacked before being fed to the model. Most models expect the targets under the
                argument :obj:`labels`. Check your model's documentation for all accepted arguments.

        Return:
            :obj:`torch.Tensor`: The tensor with training loss on this batch.
        """

        model.train()
        inputs = self._prepare_inputs(inputs)

        #if self.amp.use_amp:
        if self.use_cuda_amp:
            print('*************************************************** OK cudaAmop*********************************************')
            with autocast():
                loss = self.compute_loss(model, inputs)
        else:
            print('*************************************************** USING CPU *********************************************')
            print('*************************************************** USING CPU *********************************************')
            loss = self.compute_loss(model, inputs)

        if self.args.gradient_accumulation_steps > 1:
            print('*************************************************** OKacc *********************************************')
            loss = loss / self.args.gradient_accumulation_steps

        #if self.use_amp:
        if self.use_cuda_amp:
            print('*************************************************** OK cuda *********************************************')
            self.scaler.scale(loss).backward()
        #elif self.use_apex:
        elif self.use_apex:
            print('*************************************************** USING APEX *********************************************')
            with amp.scale_loss(loss, self.optimizer) as scaled_loss:
                scaled_loss.backward()
        elif self.deepspeed:
            print('*************************************************** USING DEEPSPEED *********************************************')
            self.deepspeed.backward(loss)
        else:
            print('*************************************************** USING CPU (loss backward)*********************************************')
            loss.backward()

        return loss.detach()

In [222]:
trainer = CTCTrainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_df,
    eval_dataset=test_df,
    tokenizer=processor.feature_extractor,
)

Using cuda_amp half precision backend


In [223]:
print(trainer.label_names)

['0', '1', '2', '3', '4', '5']


### Training

In [224]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `Wav2Vec2ForSpeechClassification.forward` and have been ignored: emotion, path. If emotion, path are not expected by `Wav2Vec2ForSpeechClassification.forward`,  you can safely ignore this message.
X:\masterarbeit_programming\venv\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1920
  Num Epochs = 1
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 2
  Total optimization steps = 240
  Number of trainable parameters = 90766470


*************************************************** OK cudaAmop*********************************************
*************************************************** OKacc *********************************************
*************************************************** OK cuda *********************************************
*************************************************** OK cudaAmop*********************************************
*************************************************** OKacc *********************************************
*************************************************** OK cuda *********************************************


Step,Training Loss,Validation Loss


*************************************************** OK cudaAmop*********************************************
*************************************************** OKacc *********************************************
*************************************************** OK cuda *********************************************
*************************************************** OK cudaAmop*********************************************
*************************************************** OKacc *********************************************
*************************************************** OK cuda *********************************************
*************************************************** OK cudaAmop*********************************************
*************************************************** OKacc *********************************************
*************************************************** OK cuda *********************************************
*******************************************

The following columns in the evaluation set don't have a corresponding argument in `Wav2Vec2ForSpeechClassification.forward` and have been ignored: emotion, path. If emotion, path are not expected by `Wav2Vec2ForSpeechClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 480
  Batch size = 4
Saving model checkpoint to /masterarbeit_programming/notebooks/content\checkpoint-10
Configuration saved in /masterarbeit_programming/notebooks/content\checkpoint-10\config.json
Model weights saved in /masterarbeit_programming/notebooks/content\checkpoint-10\pytorch_model.bin
Feature extractor saved in /masterarbeit_programming/notebooks/content\checkpoint-10\preprocessor_config.json


...

*************************************************** OK cudaAmop*********************************************
*************************************************** OKacc *********************************************
*************************************************** OK cuda *********************************************
*************************************************** OK cudaAmop*********************************************
*************************************************** OKacc *********************************************
*************************************************** OK cuda *********************************************
*************************************************** OK cudaAmop*********************************************
*************************************************** OKacc *********************************************
*************************************************** OK cuda *********************************************
*******************************************

The following columns in the evaluation set don't have a corresponding argument in `Wav2Vec2ForSpeechClassification.forward` and have been ignored: emotion, path. If emotion, path are not expected by `Wav2Vec2ForSpeechClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 480
  Batch size = 4
Saving model checkpoint to /masterarbeit_programming/notebooks/content\checkpoint-70
Configuration saved in /masterarbeit_programming/notebooks/content\checkpoint-70\config.json
Model weights saved in /masterarbeit_programming/notebooks/content\checkpoint-70\pytorch_model.bin
Feature extractor saved in /masterarbeit_programming/notebooks/content\checkpoint-70\preprocessor_config.json
Deleting older checkpoint [\masterarbeit_programming\notebooks\content\checkpoint-50] due to args.save_total_limit


*************************************************** OK cudaAmop*********************************************
*************************************************** OKacc *********************************************
*************************************************** OK cuda *********************************************
*************************************************** OK cudaAmop*********************************************
*************************************************** OKacc *********************************************
*************************************************** OK cuda *********************************************
*************************************************** OK cudaAmop*********************************************
*************************************************** OKacc *********************************************
*************************************************** OK cuda *********************************************
*******************************************

The following columns in the evaluation set don't have a corresponding argument in `Wav2Vec2ForSpeechClassification.forward` and have been ignored: emotion, path. If emotion, path are not expected by `Wav2Vec2ForSpeechClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 480
  Batch size = 4


KeyboardInterrupt: 

## Evaluation

In [ ]:
import librosa
from sklearn.metrics import classification_report

# save_path="/masterarbeit_programming/notebooks/content/data"
# trainReal_df, testReal_df=train_test_split(trainDF, test_size=0.2, random_state=101, stratify=trainDF["label"])
#
# trainRealDf=trainReal_df.reset_index(drop=True)
# testRealDf=testReal_df.reset_index(drop=True)
#
# testRealDf.to_csv(f"{save_path}/testReal.csv", sep="\t", encoding="utf-8", index=False)


In [ ]:
# Loading the created dataset using datasets
from datasets import load_dataset, load_metric

data_files = {
    "test": "/masterarbeit_programming/notebooks/content/data/test.csv",
}

dataset = load_dataset("csv", data_files=data_files, delimiter="\t", )

test_dfReal = dataset["test"]


print(test_dfReal)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device:{device}")

In [ ]:
model_name_path =  "facebook/wav2vec2-base-960h"
config = AutoConfig.from_pretrained(model_name_path,)
processor= Wav2Vec2Processor.from_pretrained(model_name_path)
model = Wav2Vec2ForSpeechClassification.from_pretrained(model_name_path).to(device)

In [ ]:
def predict(batch):
    features = processor(batch["input_values"], sampling_rate=processor.feature_extractor.sampling_rate, return_tensors="pt", padding=True)

    input_values = features.input_values.to(device)
    #attention_mask = features.attention_mask.to(device)

    with torch.no_grad():
        logits = model(input_values,
                       #attention_mask=attention_mask
                       ).logits

    pred_ids = torch.argmax(logits, dim=-1).detach().cpu().numpy()
    batch["predicted"] = pred_ids
    return batch

def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    speech_array = speech_array.squeeze().numpy()
    speech_array = librosa.resample(numpy.asarray(speech_array), sampling_rate, processor.feature_extractor.sampling_rate)

    batch["speech"] = speech_array
    return batch

In [ ]:
test_df

In [ ]:
test_df = test_df.map(speech_file_to_array_fn)

In [ ]:
test_dfReal.data

In [ ]:
result = test_df.map(predict, batched=True, batch_size=8)

In [ ]:
result[0]

In [ ]:
from datasets import Audio
import IPython.display as idp

idp.Audio(data=result[4]["input_values"], rate=target_sampling_rate)

In [ ]:
label_names = [config.id2label[i] for i in range(config.num_labels)]
label_names